In [ ]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import KerasTools as KT

import random
from io import BytesIO
import ipywidgets
import keras
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

tf.logging.set_verbosity(tf.logging.ERROR)
%matplotlib inline

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

r, c = 4, 8
idx = random.sample(range(len(train_images)), r*c)
plot_images = [train_images[i] for i in idx]
plt.figure(figsize=(12,6))
cnt=0
for i in range(r):
    for j in range(c):
        plt.subplot(r,c,cnt+1)
        plt.imshow(plot_images[cnt])
        plt.axis('off')
        cnt += 1
plt.show()

train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

train_labels = keras.utils.to_categorical(train_labels)
test_labels = keras.utils.to_categorical(test_labels)

In [ ]:
class plot_history(keras.callbacks.Callback):
    def __init__(self, loss_plt, acc_plt, mode='val', history = None):
        self.loss_plt = loss_plt
        self.acc_plt = acc_plt
        self.mode = mode
        if history is None:
            history = {'loss':[], 'val_loss':[], 'acc': [], 'val_acc':[]}
        self.history = history
    def on_epoch_end(self, epoch, logs={}):
        if self.mode == 'val':
            self.history['loss'].append(logs.get('loss'))
            self.history['val_loss'].append(logs.get('val_loss'))
            self.history['acc'].append(logs.get('acc'))
            self.history['val_acc'].append(logs.get('val_acc'))
        
        imbuf = BytesIO()

        plt.plot(range(1, len(self.history['loss'])+1), self.history['loss'], linewidth=2)
        plt.plot(range(1, len(self.history['val_loss'])+1), self.history['val_loss'], linewidth=2)
        if self.mode == 'final':
            plt.plot([epoch+1], [logs.get('val_loss')], 'rx', ms=5.0)
            plt.axvline(epoch+1, ls=':')
        plt.ylabel('Loss')
        plt.xlabel('Epochs')
        plt.savefig(imbuf)
        imbuf.seek(0)
        self.loss_plt.value = imbuf.getvalue()
        plt.clf()
        plt.plot(range(1, len(self.history['acc'])+1), self.history['acc'], linewidth=2)
        plt.plot(range(1, len(self.history['val_acc'])+1), self.history['val_acc'], linewidth=2)
        if self.mode == 'final':
            plt.plot([epoch+1], [logs.get('val_acc')], 'rx', ms=5.0)
            plt.axvline(epoch+1, ls=':')
        plt.ylabel('Accuracy')
        plt.xlabel('Steps')
        plt.savefig(imbuf)
        imbuf.seek(0)
        self.acc_plt.value = imbuf.getvalue()
        plt.clf()
        
        vt = "Validation" if self.mode == 'val' else "Test"
        print("Epoch {: 4d} | Loss: {:5.4f} Accuracy: {:5.4f} | {} Loss: {:5.4f} {} Accuracy {:5.4f}".format(
            epoch+1,logs.get('loss'),logs.get('acc'),vt,logs.get('val_loss'),vt,logs.get('val_acc')))

In [ ]:
def build_network():
    network = keras.models.Sequential(name='CIFAR_CNN')
    network.add(keras.layers.Conv2D(8, 3, activation='relu', input_shape=(32,32,3), name='conv1'))
    network.add(keras.layers.MaxPooling2D(2, name='pool1'))
    network.add(keras.layers.Conv2D(16, 3, activation='relu', input_shape=(32,32,3), name='conv2'))
    network.add(keras.layers.MaxPooling2D(2, name='pool2'))
    network.add(keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32,32,3), name='conv3'))
    network.add(keras.layers.MaxPooling2D(2, name='pool3'))
    network.add(keras.layers.Flatten(name='flatten'))
    network.add(keras.layers.Dropout(0.25, name='dropout'))
    network.add(keras.layers.Dense(64, activation='relu', name='dense1'))
    network.add(keras.layers.Dense(10, activation='softmax', name='output'))

    network.compile(optimizer=keras.optimizers.adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    return network

In [ ]:
network = build_network()
network.summary()

loss_plt = ipywidgets.Image()
acc_plt = ipywidgets.Image()
display(ipywidgets.HBox([loss_plt, acc_plt]))

epochs = 50
history = network.fit(train_images, train_labels, epochs=epochs, 
                      batch_size=128, validation_split=0.25, verbose=0, callbacks=[plot_history(loss_plt, acc_plt)])

In [ ]:
network = build_network()
network.summary()

loss_plt = ipywidgets.Image()
acc_plt = ipywidgets.Image()
display(ipywidgets.HBox([loss_plt, acc_plt]))

epochs = 25
network.fit(train_images, train_labels, epochs=epochs, 
            batch_size=128, validation_data=((test_images, test_labels)), verbose=0, 
            callbacks=[plot_history(loss_plt, acc_plt, mode='final', history=history.history)])

In [ ]:
entry=9
ground_truth = KT.datasets.decode.decode_dataset('cifar10', np.argmax(test_labels[entry]))

prediction = network.predict(test_images[entry:entry+1])
predicted_label = KT.datasets.decode.decode_dataset('cifar10',np.argmax(prediction, axis=1)[0])

img = test_images[entry]
plt.figure(figsize=(2,2))
plt.imshow(img)
plt.show()

print("Ground truth: {} | Predicted: {}".format(ground_truth,predicted_label))

decoded_prediction = KT.datasets.decode.decode_predictions('cifar10', prediction, top=10)
results = [{'label': l, 'score': s*100} for i,l,s in decoded_prediction[0]]
print('-' * 32)
print("{0:>12} | {1:>6} ".format("Label", "Score"))
print('-' * 32)
for r in results:
    print("{label:>12} | {score:>6.2f}% ".format(**r))
print('-' * 32)